In [1]:
import pandas as pd
import numpy as np
from pandas.testing import assert_frame_equal
import path
import xlsxwriter

In [2]:
lan_thi = '1'

dtype = {
    'Mã HV': 'str',
    'Họ': 'str',
    'Tên': 'str',
    'CC': 'float',
    'GHP': 'float',
    'Thi_LT': 'str',
    'Thi_TH': 'str',
    'Điểm HP': 'str',
    'Ghi chú': 'str'
}

df1 = pd.read_excel(
    path.INPUT, skiprows = 9, 
    usecols = [*dtype], dtype=dtype
).rename(columns={
    'Mã HV': 'user_id',
    'Họ': 'name_last',
    'Tên': 'name_first',
    'CC': 'CC',
    'GHP': 'QT',
    'Thi_LT': f'THILT{lan_thi}_PRINT',
    'Thi_TH': f'THITH{lan_thi}_PRINT',
    'Điểm HP': f'TB{lan_thi}',
    'Ghi chú': f'NOTE{lan_thi}'
})

df1 = df1.loc[df1['user_id'].notna()].fillna('')
# df2 = pd.read_excel('XtraReport_BangDiemThiCotDong.xls', skiprows = 9)

In [3]:
df1

,user_id,name_last,name_first,CC,QT,THILT1_PRINT,THITH1_PRINT,TB1,NOTE1
0,2155010001,LÊ HUY MINH,ANH,9.8,9.0,9.0,9.5,9.2,
1,2155010002,LÊ NGUYỄN TRÂM,ANH,10.0,8.5,7.0,9.5,8.3,
2,2155010003,DƯƠNG NGỌC,ÁNH,10.0,8.0,8.5,9.5,8.8,
3,2155010004,NGÔ HỒNG,BẢO,10.0,9.0,9.0,10.0,9.4,
4,2155010005,LÂM NGỌC,BÍCH,10.0,9.0,10.0,10.0,9.8,
...,...,...,...,...,...,...,...,...,...
82,2155010080,BÙI THỊ CẨM,VÂN,0.0,8.5,8.5,10.0,8.1,
83,2155010081,PHÙNG HẢI,VÂN,10.0,10.0,10.0,10.0,10,
84,2155010087,XAYAVONG,VIYADA,9.0,9.0,6.5,8.0,7.7,
85,2155010082,DƯƠNG LỤC ÁNH,XUÂN,6.7,9.0,5.5,4.0,5.9,


In [4]:
def write_bang_diem_INFO1001(df, path, footer='',
                         mon='', lop='', lop_hp='', lan_thi='', ngay_thi='',
                         ngay='      ', thang='03', nam='2022'):
    table_row = df.shape[0]
    workbook = xlsxwriter.Workbook(path)
    worksheet = workbook.add_worksheet()
    worksheet.set_portrait()
    worksheet.set_paper(9)  # A4
    worksheet.center_horizontally()
    worksheet.set_margins(left=0.7, right=0.7, top=0.75, bottom=0.75)
    worksheet.repeat_rows(10)
    worksheet.print_area(0, 0, 10 + table_row + 21, 9)
    worksheet.set_print_scale(75)
    worksheet.set_footer(f'&C&P/&N&R{footer}')
    worksheet.set_column(0, 0, 4)
    worksheet.set_column(1, 1, 12)
    worksheet.set_column(2, 2, 28)
    worksheet.set_column(3, 3, 12)
    worksheet.set_column(4, 8, 8)
    worksheet.set_column(9, 9, 14)
    worksheet.set_row(3, 8)
    align_center = workbook.add_format({'align': 'center'})
    bold = workbook.add_format({'bold': True})
    header_format = workbook.add_format({'align': 'center', 'bold': True, 'border': 1})
    mark_format = workbook.add_format({'num_format': '0.0', 'border': 1, 'align': 'right'})
    cell_format_center = workbook.add_format({'border': 1, 'align': 'center'})
    cell_format_left = workbook.add_format({'border': 1, 'align': 'left'})

    # Metadata
    worksheet.merge_range(
        'A1:J1',
        'TRƯỜNG ĐẠI HỌC Y KHOA PHẠM NGỌC THẠCH',
        workbook.add_format({'align': 'center'}))

    worksheet.merge_range(
        'A2:J2',
        'KHOA Y TẾ CÔNG CỘNG',
        workbook.add_format({'align': 'center'}))
    
    worksheet.merge_range(
        'A3:J3',
        'BỘ MÔN TIN HỌC - THỐNG KÊ Y HỌC',
        workbook.add_format({'align': 'center', 'bold': True}))

    worksheet.merge_range(
        'A4:J4',
        '\u23AF' * 8,
        workbook.add_format({'align': 'center', 'bold': True}))

    worksheet.merge_range(
        'A5:J5',
        'BẢNG ĐIỂM',
        workbook.add_format({'align': 'center', 'bold': True}))

    worksheet.merge_range(
        'A6:J6',
        'MÔN HỌC: ' + mon,
        workbook.add_format({'align': 'center', 'bold': True}))

    worksheet.merge_range(
        'A7:J7',
        'MÃ LỚP HỌC PHẦN: ' + lop_hp,
        workbook.add_format({'align': 'center', 'bold': True}))

    worksheet.merge_range(
        'A8:J8',
        'MÃ LỚP: ' + lop,
        workbook.add_format({'align': 'center', 'bold': True}))

    worksheet.merge_range(
        'A9:J9',
        'LẦN THI: ' + lan_thi,
        workbook.add_format({'align': 'center', 'bold': True}))

    worksheet.write(
        'A10',
        'Ngày thi: ' + ngay_thi,
        workbook.add_format({'align': 'left', 'italic': True}))
    
    # Header
    worksheet.write(
        'A11',
        'STT',
        header_format)

    worksheet.write(
        'B11',
        'MSSV',
        header_format)

    worksheet.merge_range(
        'C11:D11',
        'Họ và tên',
        header_format)

    worksheet.write(
        'E11',
        'CC',
        header_format)

    worksheet.write(
        'F11',
        'TX(1)',
        header_format)

    worksheet.write(
        'G11',
        'Thi#LT',
        header_format)
    
    worksheet.write(
        'H11',
        'Thi#TH',
        header_format)
    
    worksheet.write(
        'I11',
        'TB',
        header_format)

    worksheet.write(
        'J11',
        'Ghi chú',
        header_format)

    # write data
    worksheet.write_column('A12', df.index + 1, cell_format_center)
    worksheet.write_column('B12', df['user_id'], cell_format_center)
    worksheet.write_column('C12', df['name_last'], cell_format_left)
    worksheet.write_column('D12', df['name_first'], cell_format_left)
    worksheet.write_column('E12', df['CC'], mark_format)
    worksheet.write_column('F12', df['QT'], mark_format)
    worksheet.write_column('G12', df[f'THILT{lan_thi}_PRINT'], mark_format)
    worksheet.write_column('H12', df[f'THITH{lan_thi}_PRINT'], mark_format)
    worksheet.write_column('I12', df[f'TB{lan_thi}'], mark_format)
    worksheet.write_column('J12', df[f'NOTE{lan_thi}'].fillna(''), cell_format_left)
    
    # Tinh tong, du thi, dau, rot, vang
    tong_so = table_row
    vang = df[(df[f'TB{lan_thi}'] == 'VT')].shape[0]
    du_thi = tong_so - vang
    try:
        rot_lt = df[f'NOTE{lan_thi}'].value_counts(dropna=False)['Rớt LT']
    except:
        rot_lt = 0
    try:
        rot_th = df[f'NOTE{lan_thi}'].value_counts(dropna=False)['Rớt TH']
    except:
        rot_th = 0
    try:
        rot_lt_th = df[f'NOTE{lan_thi}'].value_counts(dropna=False)['Rớt LT, TH']
    except:
        rot_lt_th = 0
    try:
        rot_tb = df[f'NOTE{lan_thi}'].value_counts(dropna=False)['Rớt']
    except:
        rot_tb = 0
    try:
        dau = df[f'NOTE{lan_thi}'].value_counts(dropna=False)['']
    except:
        dau = 0
    # print(df[f'NOTE{lan_thi}'].value_counts(dropna=False))
    rot = rot_lt + rot_th + rot_lt_th + rot_tb
    
    # write data
    worksheet.write(
        f'B{10 + df.shape[0] + 3}', 
        f'Tổng số: ',
        workbook.add_format({'align': 'right', 'italic': True}))
    worksheet.write(
        f'C{10 + table_row + 3}', 
        tong_so,
        workbook.add_format({'align': 'left', 'italic': True}))
    worksheet.write(
        f'B{10 + table_row + 4}', 
        f'Dự thi: ',
        workbook.add_format({'align': 'right', 'italic': True}))
    worksheet.write(
        f'C{10 + table_row + 4}', 
        du_thi,
        workbook.add_format({'align': 'left', 'italic': True}))
    worksheet.write(
        f'B{10 + table_row + 5}', 
        f'Đậu: ',
        workbook.add_format({'align': 'right', 'italic': True}))
    worksheet.write(
        f'C{10 + table_row + 5}', 
        dau,
        workbook.add_format({'align': 'left', 'italic': True}))
    worksheet.write(
        f'B{10 + table_row + 6}', 
        f'Rớt: ',
        workbook.add_format({'align': 'right', 'italic': True}))
    worksheet.write(
        f'C{10 + table_row + 6}', 
        rot,
        workbook.add_format({'align': 'left', 'italic': True}))
    worksheet.write(
        f'B{10 + table_row + 7}', 
        f'Vắng: ',
        workbook.add_format({'align': 'right', 'italic': True}))
    worksheet.write(
        f'C{10 + table_row + 7}', 
        vang,
        workbook.add_format({'align': 'left', 'italic': True}))
    
    worksheet.merge_range(
        f'E{10 + table_row + 8}:J{10 + table_row + 8}',
        f'Tp. Hồ Chí Minh, ngày {ngay} tháng {thang} năm {nam}',
        workbook.add_format({'align': 'center', 'italic': True}))
    
    worksheet.merge_range(
        f'A{10 + table_row + 9}:D{10 + table_row + 9}',
        'GIÁO VỤ BỘ MÔN',
        workbook.add_format({'align': 'center', 'bold': True}))
    
    worksheet.merge_range(
        f'A{10 + table_row + 14}:D{10 + table_row + 14}',
        'BS. Phạm Hoàng Gia Khương',
        workbook.add_format({'align': 'center', 'bold': True}))
    
    worksheet.merge_range(
        f'E{10 + table_row + 9}:J{10 + table_row + 9}',
        'TRƯỞNG BỘ MÔN',
        workbook.add_format({'align': 'center', 'bold': True}))
    
    worksheet.merge_range(
        f'E{10 + table_row + 14}:J{10 + table_row + 14}',
        'TS. BS. Nguyễn Ngọc Vân Phương',
        workbook.add_format({'align': 'center', 'bold': True}))
    
    worksheet.merge_range(
        f'A{10 + table_row + 17}:D{10 + table_row + 17}',
        'PHÒNG QUẢN LÝ ĐÀO TẠO ĐẠI HỌC',
        workbook.add_format({'align': 'center', 'bold': True}))
    
    workbook.close()


In [5]:
write_bang_diem_INFO1001(
    df1, path.OUTPUT, footer='INFO1001-2021-YRD-1',
    mon='Thống kê y học [2 TC]', lop='RHM-2021', lop_hp='211TKY_01001203', lan_thi='1', ngay_thi='14/04/2022',
    ngay='      ', thang='04', nam='2022')

In [47]:
# drop rows and columns
df1.dropna(axis = 0, thresh = 5, inplace = True)
df2.dropna(axis = 0, thresh = 5, inplace = True)
df2 = df2[df2.columns.drop(list(df2.filter(regex='Unnamed')))]

In [32]:
# change score's type int to float
# score's type now is float in spite of value 'VT' in column
df1.replace('VT', 1000, inplace = True)
df1.replace(1000, 'VT', inplace = True)

In [33]:
convert_dtype_df1 = {
#                             'STT': 'int',
                          'MSSV': 'float',
#                            'CC': 'float',
#                            'Thi#LT': 'float',
#                            'Thi#TH': 'float',
}
df1 = df1.astype(convert_dtype_df1)

# change all columns into string
convert_float_df1 = {'CC': 'string',
                     'TX(1)': 'string',
                    'Thi#LT': 'string',
#                     'Thi#TH': 'string',
                     'TB': 'string',
                     'STT': 'string',
                     'MSSV': 'string',
                     'Họ và tên': 'string',
                    'Unnamed: 3': 'string',
}
df1 = df1.astype(convert_float_df1)

In [34]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   STT         32 non-null     string 
 1   MSSV        32 non-null     string 
 2   Họ và tên   32 non-null     string 
 3   Unnamed: 3  32 non-null     string 
 4   CC          32 non-null     string 
 5   TX(1)       32 non-null     string 
 6   Thi#LT      32 non-null     string 
 7   TB          32 non-null     string 
 8   Ghi chú     0 non-null      float64
dtypes: float64(1), string(8)
memory usage: 2.5 KB


In [35]:
# score's type is int because of value 'VT' in column 'Diem HP'
# change score's type int to float
df2.replace('VT', 1000, inplace = True)
df2.replace(1000, 'VT', inplace = True)
df2.fillna('VT', inplace = True)

In [36]:
# convert_dtype_df2 = {
#     'STT': 'int',
#     'Mã HV': 'float',
#     'Họ': 'string',
#     'Tên': 'string',
#     'CC': 'float',
#     'Thi_LT': 'float',
# #     'Thi_TH': 'float',
#     'GHP': 'float',
#     'Điểm HP': 'float'
# }
# df2 = df2.astype(convert_dtype_df2)

In [37]:
convert_float_df2 = {
    'STT': 'string',
    'Mã HV': 'string',
    'Họ': 'string',
    'Tên': 'string',
    'CC': 'string',
    'Thi_LT': 'string',
#     'Thi_TH': 'string',
    'GHP': 'string',
    'Điểm HP': 'string'
}
df2 = df2.astype(convert_float_df2)

In [38]:
df_1 = df1.drop(['Ghi chú', 'STT'], axis = 1)
df_2 = df2.drop(['P', 'Ngày sinh', 'Ghi chú', 'STT'], axis = 1)

In [39]:
df_1 = df_1.sort_values('MSSV')
df_2 = df_2.sort_values('Mã HV')

In [40]:
df_1 = df_1.set_index('MSSV')
df_2 = df_2.set_index('Mã HV')

In [41]:
df_1.columns= df_2.columns

In [42]:
assert_frame_equal(df_1, df_2, check_names = False, check_exact = True)

In [43]:
for i in range(df_1.shape[1]):
    try:
        pd.testing.assert_frame_equal(
            pd.DataFrame(df_1.iloc[:, i]), pd.DataFrame(df_2.iloc[:, i]), check_names = False, check_exact = True
        )
    except AssertionError as e:
        print(e, "\n")

In [44]:
try:
    pd.testing.assert_frame_equal(df_1, df_2, check_names=False, check_less_precise=True)
except AssertionError as e:
    print(e)
    df = (df_1 == df_2)
    print(df_1 == df_2)
    raise

C:\Users\ASUS\AppData\Local\Temp/ipykernel_4044/1488617621.py:2: FutureWarning: The 'check_less_precise' keyword in testing.assert_*_equal is deprecated and will be removed in a future version. You can stop passing 'check_less_precise' to silence this warning.
  pd.testing.assert_frame_equal(df_1, df_2, check_names=False, check_less_precise=True)
